# Распознавание объектов на изображениях с претренированной сетью 




Грузим библиотеки (точно нужны слои,  optimizer , cifar10)

In [0]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import  BatchNormalization
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense


from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline 

from tensorflow.keras import utils
# Количество классов изображений
nb_classes = 10
# Названия классов из набора данных CIFAR-10
classes=['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train10 = utils.to_categorical(y_train, nb_classes)

y_test10 = utils.to_categorical(y_test, nb_classes)

#x_train=np.broadcast_to(x_train[..., None],(60000,28,28,3))
#x_test=np.broadcast_to(x_test[..., None],(10000,28,28,3))

задаем размер входа

In [0]:
# Размер изображений  CIFAR10
img_width, img_height, chanel_n = 32, 32, 3
#img_width, img_height, chanel_n = 28, 28, 3

print(x_train.shape)
#пакет для генератора
batch_size = 128


#Загрузка VGG16

In [0]:
# Загружаем сеть VGG16 без части, которая отвечает за классификацию
base_model = applications.VGG16(weights ='imagenet', include_top=False, input_shape=(img_width, img_height, chanel_n ))

base_model.summary()


Делаем генератор преобразования объекта - только масштаб значений

In [0]:
# Создаем генератор данных для обучения
datagen = ImageDataGenerator(rescale=1. / 255)



train_generator = datagen.flow(
    x_train,y_train10,
    batch_size=batch_size,
    )
# Создаем генератор данных для валидации
val_generator =  datagen.flow(
    x_test,y_test10,
    batch_size=batch_size,
    )

test_generator = datagen.flow(
    x_test,y_test10,
    batch_size=batch_size,
    )

#Свой классификатор поверх VGG16 

In [0]:
from tensorflow.keras.layers import BatchNormalization

# Замораживаем сеть VGG16
base_model.trainable = False

models = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
models.add(base_model) # только свертка
models.add(BatchNormalization())
models.add(Flatten()) # векторим вход
models.add(Dense(100,activation='relu'))
models.add(Dense(10,activation='softmax'))


models.summary()


print(models.layers[-1].input_shape)

print(models.layers[-1].output_shape)


Пропускаем выборку через VGG16 свертки

In [0]:

predict1=models.predict(x_train[:20])
print(predict1.shape)

Посмотрим, что вышло

In [0]:
print(predict1.shape)

plt.plot(predict1.mean(axis=0))
plt.xlabel('feature number')
plt.ylabel('mean feature value')
plt.title('predict по VGG16')
plt.show()

plt.subplot(3,1,1)
plt.plot(predict1[10,:])
plt.plot(y_train10[10,:],'r')
plt.xlabel('feature number')
plt.ylabel('feature value')
plt.title('sample 10 по VGG16')
plt.show()
plt.subplot(3,1,2)
plt.plot(predict1[1,:])
plt.plot(y_train10[1,:],'r')
plt.xlabel('feature number')
plt.ylabel('feature value')
plt.title('sample 1 по VGG16')
plt.show()
plt.subplot(3,1,3)
plt.plot(predict1[5,:])
plt.plot(y_train10[5,:],'r')
plt.xlabel('feature number')
plt.ylabel('feature value')
plt.title('sample 5 по VGG16')

plt.show()
ik5=np.argmax(predict1[10,:])
ik1=np.argmax(predict1[1,:])
ik0=np.argmax(predict1[5,:])
print(ik5,ik1,ik0)
print(y_train[[10,1,5]])


Видны какие-то "левые" реакции будем исправлять выходной массив


In [0]:
# компилияция модели
models.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=["accuracy"])

In [0]:
hist = models.fit(x=x_train,y=y_train10, epochs=10, batch_size=128, validation_data=(x_test, y_test10), verbose=1)

test_score = models.evaluate(x_test, y_test10)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

In [0]:
plt.plot(hist.history['val_acc'],'r',label='val_accuracy')
plt.plot(hist.history['acc'],'g',label='train_accuracy')
plt.legend()
plt.show()

In [0]:
predict2=predict1.copy() 

predict1=models.predict(x_train[:20])
print(predict1.shape)

In [0]:
print(predict1.shape)

plt.plot(predict1.mean(axis=0))
plt.xlabel('feature number')
plt.ylabel('mean feature value')
plt.title('predict по VGG16')
plt.show()

plt.subplot(3,1,1)
plt.plot(predict1[10,:])
plt.plot(y_train10[10,:],'r')
plt.xlabel('feature number')
plt.ylabel('feature value')
plt.title('sample 10 по VGG16')
plt.show()
plt.subplot(3,1,2)
plt.plot(predict1[1,:])
plt.plot(y_train10[1,:],'r')
plt.xlabel('feature number')
plt.ylabel('feature value')
plt.title('sample 1 по VGG16')
plt.show()
plt.subplot(3,1,3)
plt.plot(predict1[5,:])
plt.plot(y_train10[5,:],'r')
plt.xlabel('feature number')
plt.ylabel('feature value')
plt.title('sample 5 по VGG16')

plt.show()
ik5=np.argmax(predict1[10,:])
ik1=np.argmax(predict1[1,:])
ik0=np.argmax(predict1[5,:])
print(ik5,ik1,ik0)
print(y_train[[10,1,5]].T)

Посмотрим , что распознается

In [0]:
n=1

plt.imshow(x_train[n][:,:,:])
plt.title(classes[np.argmax(predict1[n,:])])
plt.show()

In [0]:
predict1_test=models.predict(x_test[:20])


Сделаем немного тестов

In [0]:
n=5

plt.imshow(x_test[n][:,:,:])
plt.title(classes[np.argmax(predict1_test[n,:])])
plt.show()